##  Optimization manual steps

In [1]:
import random
import numpy as np
import pprint
from mycrograd_debug.engine_debug import Value
from mycrograd_debug.nn_debug import MLP
from mycrograd_debug.drawviz_debug import draw_dot, draw_nn
from mycrograd_debug.util_debug import (
    debugPrint,
    print_my_params,
    print_all_values,
    backupParameters,
    restoreParameters,
)
import os

np.random.seed(1337)
random.seed(1337)
number_of_iterations = 10
pp = pprint.PrettyPrinter(indent=4)
global activation
global loss
global step
step=0

makeimg = False
# debugOptions={"params"}
debugOptions={}

In [2]:
path = "images/nn"
isExist = os.path.exists(path)
print(isExist)
if not isExist:
    os.makedirs(path)
    print("dir created")


True


#### Multi Layer Perceptron 1-2-1

In [3]:
# initialize a model
nin = 1  # number of inputs
nout = 1  # number of outputs
Value.value_counter = 0
model = MLP(nin, [2, nout], weightsinit=2, lastReLU=False, debug_bw=False)
# xinumbers = list(range(4, 4 + nin))
xinumbers = [4]
xinput = [Value(x, type="i%s" % index) for index, x in enumerate(xinumbers, start=1)]
xtarget = Value(1.2, type="t")  # desired targets
# xtarget = Value(0.0, type="t")  # desired targets
debugPrint(model, {"params"}, message="start", inputs=xinput, targets=xtarget)

Module nn MLP: structure [1, 2, 1]
start
parameters
 name lay neu ty   data   grad
 v001  L1  N1 w1   0.50   0.00
 v002  L1  N1  b   0.00   0.00
 v003  L1  N2 w1   0.50   0.00
 v004  L1  N2  b   0.00   0.00
 v005  L2  N1 w1   0.60   0.00
 v006  L2  N1 w2   1.20   0.00
 v007  L2  N1  b   0.00   0.00
inputs
[Value(name=v008,layernumber=,neuronnumber=,weightnumber=,type=i1,data=4, grad=0)]
targets
Value(name=v009,layernumber=,neuronnumber=,weightnumber=,type=t,data=1.2, grad=0)


In [4]:
# loss function single MLP
def loss_single(activation, target):
    total_loss = (activation - target) * (activation - target)
    # total_loss = activation*activation
    total_loss.type = "l"
    return total_loss


In [5]:
def imageFunc(filename):
    # print(filename)
    # print("loss",loss)
    if makeimg:
        dot1=draw_dot(loss)
        dot1.render("images/"+filename , format="svg", view=True)
        dot2=draw_nn(xinput, model)
        dot2.node(
            name="loss", label="step %2d loss %6.2f" % (step, loss.data), shape="record"
        )
        dot2.render("images/nn/"+filename, format="svg", view=True)

In [6]:
def getactivation(filename="default"):
    #### forward pass0
    global model
    global loss
    global activation
    global step
    step = step + 1
    activation = model(xinput)
    loss = loss_single(activation, xtarget)
    # loss = activation*activation
    # loss.type="l"
    debugPrint(model, debugOptions, message="act")
    imageFunc("step%d_1loss" % step)


def zeroGradients(filename="default"):
    global model
    global step
    model.zero_grad()
    for i in xinput:
        i.grad = 0
    # print("zero'd gradients")
    debugPrint(model, debugOptions, message="zer")
    imageFunc("step%d_2zero" % step)


def backward(filename="default"):
    #### backward pass
    global activation
    global step
    loss.backward()
    # print("parameters after backpass")
    debugPrint(model, debugOptions, message="bwd")
    imageFunc("step%d_3back" % step)


def updateParams(filename="default"):
    #### update
    global model
    global step
    for p in model.parameters():
        p.data += -0.05 * p.grad
    # print("updated parameters")
    debugPrint(model, debugOptions, message="upd")
    imageFunc("step%d_3upda" % step)


In [7]:
def optStep(filename="default"):
    global model
    global step
    getactivation()
    zeroGradients()
    backward()
    updateParams()
    print(f"step %3d output %6.4f loss %6.8f" % (step, activation.data, loss.data))
    dot = draw_nn(xinput, model)
    dot.render("static/" + filename)


In [8]:
def opt():
    global loss

    for k in range(number_of_iterations):
        # print("start step %d" % k)
        optStep(filename="default")
        # print(f"step %3d output %6.4f loss %6.4f" % (k, activation.data, loss.data))

opt()

step   1 output 3.6000 loss 5.76000000
step   2 output -2.3789 loss 12.80838205
step   3 output -0.3633 loss 2.44380049
step   4 output 0.2206 loss 0.95927443
step   5 output 0.3652 loss 0.69687608
step   6 output 0.4730 loss 0.52852836
step   7 output 0.5689 loss 0.39831707
step   8 output 0.6606 loss 0.29094950
step   9 output 0.7510 loss 0.20164261
step  10 output 0.8397 loss 0.12982311
